In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyer_loader import JupyerLoader

loader = JupyerLoader()
loader.load("s04_engine_interface")
loader.load("s05_random_engine")
loader.load("s06_play")

# Evaluation

In order to pick the best move instead of just a random one, our engine needs some way to evaluate chess positions. To this end, we will define a basic interface `Evaluator` and a concrete implementation `PieceEvaluator` that will be capable of two things:
* Evaluate terminal state positions
* Evaluate all other position by using a heursitic

As the code will be more complex than before, classes will not be shown as one block of code, but rather methods and attributes will be added dynamically one after another. This form of dynamic class modification is called [monkey patching](https://docs.plone.org/appendices/glossary.html#term-Monkey-patch) in the python community and is often used as a technique to patch third party code.

To evaluate terminal states, we first need to define what a terminal state ist. These are all the end positions of finished games. 
A finished game is either won by one of the two players or a draw. To check whether the game is won, the method `is_checkmate()` of the `chess.Board` class can be used. 
For a draw there are multiple functions vor various condiditions like `is_stalemate` or `is_insufficient_material`. 
To make the code more expressive a new function `is_draw` is added to the `chess.Board` class that checks for any of these conditions.

In [ ]:
import chess


def is_draw(self) -> bool:
    return self.is_stalemate() or self.is_insufficient_material(
    ) or self.is_fivefold_repetition() or self.is_seventyfive_moves()


chess.Board.is_draw = is_draw

These two methods allows us to implement the basic interface `Evaluator`. The constructor takes two parameters `value_checkmate` and `value_draw` which denote the score returned for a checkmate and draw, respectively. The method `evaluate` takes only the board `board` as a parameter and returns the score of the game if it is finished and `None` otherwise. The second method is `evaluate` which has the same signature as `evaluate` and evaluates unfinished game positions. This method will be provided by concrete subclasses.

In [ ]:
class Evaluator():

    def __init__(self, value_checkmate: int, value_draw: int):
        self.value_checkmate = value_checkmate
        self.value_draw = value_draw

    def evaluate(self, board: chess.Board) -> int:
        if board.is_checkmate():
            return -self.value_checkmate
        if board.is_draw():
            return self.value_draw
        return None

    def heuristic(self, board: chess.Board) -> int:
        pass

Next, a concrete subclass `PieceEvaluator` is implemented that provides the `heursitic`. This simple heuristic will calculate the value of the position based on the material on the board. The class inherits from the previously defined class `Evaluator`. The constructor will also take `value_checkmate` and `value_draw` that are passed to the superclass. Additionally a parameter `piece_values` is needed, which is a dict that assigns each piece type a value.

In [ ]:
class PieceEvaluator(Evaluator):

    def __init__(
        self, piece_values: dict, value_checkmate: int, value_draw: int
    ):
        super().__init__(value_checkmate, value_draw)
        self.piece_values = piece_values

We can then provide the implementation for the `heuristic` method. The evaluation of the game that is not finished is determined by summing up the piece values of the player to move
and subtracting the piece values of the opponent. A positive score therefore is good for player to move whereas a negative score is good for the opponent. To effiently iterate over all squares with pieces, the board representation `board.occupied_co` can be used together with the `scan_reversed` function of python-chess.

In [ ]:
def heuristic(self, board: chess.Board) -> int:
    playerScore = 0
    for square in chess.scan_reversed(board.occupied_co[board.turn]):
        playerScore += self.piece_values[board.piece_type_at(square)]

    opponentScore = 0
    for square in chess.scan_reversed(board.occupied_co[not board.turn]):
        opponentScore += self.piece_values[board.piece_type_at(square)]

    return playerScore - opponentScore


PieceEvaluator.heuristic = heuristic

To actually create an instance of the `PieceEvaluator` we need to define scores for a checkmate, draw and each piece type. A very common [standard piece value assignment](https://en.wikipedia.org/wiki/Chess_piece_relative_value) is used.

In [ ]:
STANDARD_PIECE_VALUES = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0
}

STANDARD_VALUE_CHECKMATE = 100
STANDARD_VALUE_DRAW = 0

Then the instance can be created with these values.

In [ ]:
standard_evaluator = PieceEvaluator(
    piece_values=STANDARD_PIECE_VALUES,
    value_checkmate=STANDARD_VALUE_CHECKMATE,
    value_draw=STANDARD_VALUE_DRAW
)

We can write some tests for the evaluator to make sure it is working correctly. The first test checks whether `evaluate` correctly detects a mate.

In [ ]:
import IPython.display

SCHOLAR_MATE = "r1bqkb1r/pppp1Qpp/2n2n2/4p3/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 0 4"
board = chess.Board(SCHOLAR_MATE)
IPython.display.display(board)

# It is blacks turn, but he was checkmated. Therefore the score should be the worst possible -100
score = standard_evaluator.evaluate(board)
assert score == -100, f"{score} != {-100}"

The second case will be tested with a stalemate position. In this case the `evaluate` method should return the score for a draw.

In [ ]:
# Viktor Korchnoi vs Anatoly Karpov, World Champtionchip 5th game 1987
STALE_MATE = "8/5KBk/8/8/p7/P7/8/8 b - - 34 124"
board = chess.Board(STALE_MATE)
IPython.display.display(board)

# It is a stalemate, so the score should be zero
score = standard_evaluator.evaluate(board)
assert score == 0, f"{score} != {0}"

The last case to be tested is a game that is not finished. The `evaluate` method should return `None` and instead `heuristic` should calculate the score.

In [ ]:
# Topalov, Veselin (2740) vs. Shirov, Alexei (2710)
SHIROV_SACRIFICE = "8/8/4kpp1/3p1b2/p6P/2B5/6P1/6K1 b - - 0 47"
board = chess.Board(SHIROV_SACRIFICE)
IPython.display.display(board)

# Game not finished, so score should be None
score = standard_evaluator.evaluate(board)
assert score == None, f"{score} != None"

# Black has 2 points more and is about to move, so the score should be 2
score = standard_evaluator.heuristic(board)
assert score == 2, f"{score} != 2"

To make sure the `heursitic` method is working correctly, it will be tested on another game.

In [ ]:
# Evgeny Yuryevich Vladimirov vs. Vladimir Viktorovich Epishin
VLADIMIROV_THUNDERBOLT = "r4k1r/1b2bPR1/p4n2/3p4/4P2P/1q2B2B/PpP5/1K4R1 w - - 0 26"
board = chess.Board(VLADIMIROV_THUNDERBOLT)
IPython.display.display(board)

# Game not finished, so score should be None
score = standard_evaluator.evaluate(board)
assert score == None, f"{score} != None"

# Black has 10 points more, but White is about to move, so the score should be -10
score = standard_evaluator.heuristic(board)
assert score == -10, f"{score} != -10"

To use this newly created `Evaluator` an `EvaluationEngine` is created
and expects an instance of an `Evaluator` to be passed as an argument.
The `EvaluationEngine` inherits from the `Engine` class.

In [ ]:
import random
import chess.engine
from converted_notebooks.s04_engine_interface import Engine, ScoredMove


class EvaluationEngine(Engine):

    def __init__(self, evaluator: Evaluator):
        self.evaluator = evaluator

As the `Evaluator` calculates the value from the current players point of view, while it is customary to express the valuation from Whites point, a `PLAYER_MULTIPLIER` dict is used to translate between absolute and relative values.

In [ ]:
PLAYER_MULTIPLIER = {
    chess.WHITE: 1, chess.BLACK: -1
}

EvaluationEngine.PLAYER_MULTIPLIER = PLAYER_MULTIPLIER

To get the value of a possible move, 
we define a method `_evaluate_move` 
which takes the instance of the `EvaluationEngine` class, 
and a chess board `board`,
and a chess move `move`
as parameters
and returns a `ScoredMove`.

After making the move
it uses the class' `evaluator` instance to check 
if the board has a final score using the `evaluate` method
and determines the score of the board using the `heuristic` if it hasn't.

To turn the score into an absolute value it is multiplied with the class' `PLAYER_MULTIPLIER`.

Afterwards the board is returned to the initial state before returning the `ScoredMove`.

In [ ]:
def _evaluate_move(self, board: chess.Board, move: chess.Move) -> ScoredMove:
    board.push(move)
    score = self.evaluator.evaluate(board)
    if score is None:
        score = self.evaluator.heuristic(board)
    score *= self.PLAYER_MULTIPLIER[board.turn]
    board.pop()

    return ScoredMove(score, move)


EvaluationEngine._evaluate_move = _evaluate_move

To get the value of all legal moves,
we define the method `_evaluate_moves`
which takes the instance of the `EvaluationEngine` class
and a chess board `board` as parameters.
It returns a list of `ScoredMove` objects by evaluating all legal moves.

In [ ]:
def _evaluate_moves(self, board: chess.Board):
    return [self._evaluate_move(board, move) for move in board.legal_moves]


EvaluationEngine._evaluate_moves = _evaluate_moves

Next, the `analyse` method can be defined using the `_evaluate_moves` method.

The scored moves are then shuffled and, afterwards, sorted by their score, 
creating a different order of moves having the same score, 
depending on the state of the RNG.

By default, the python `sort` method sorts from lowest to highest. 
Therefore, the first move is the best for Black, unless the sorting order is reversed.

In [ ]:
def analyse(self, board: chess.Board) -> list[ScoredMove]:
    nextMoves = self._evaluate_moves(board.copy(stack=False))
    random.shuffle(nextMoves)

    whitesTurn = board.turn is chess.WHITE
    nextMoves.sort(reverse=whitesTurn)

    return nextMoves


EvaluationEngine.analyse = analyse

At last the abstract `play` method needs to be implemented.
This is done by taking the best scored move as determined by the `analyse` method 
and returning it as a `chess.engine.PlayResult`.


In [ ]:
def play(self, board: chess.Board) -> chess.engine.PlayResult:
    bestScoredMove = self.analyse(board)[0]
    return chess.engine.PlayResult(move=bestScoredMove.move, ponder=None)


EvaluationEngine.play = play

As before we can let the new defined engine play against the `RandomEngine`. Through the use of the evaluation function it should already play signifcantly better and win most of the games.

In [ ]:
import random
from converted_notebooks.s06_play import play_game
from converted_notebooks.s05_random_engine import RandomEngine

random.seed(42)

board = chess.Board()
play_game(board, EvaluationEngine(evaluator=standard_evaluator), RandomEngine())
print(board.outcome())